In [2]:
import sys
sys.path.insert(1, '../')
print(sys.path)

['/home/cboned/miniconda3/envs/graphocr/lib/python310.zip', '../', '/home/cboned/miniconda3/envs/graphocr/lib/python3.10', '/home/cboned/miniconda3/envs/graphocr/lib/python3.10/lib-dynload', '', '/home/cboned/miniconda3/envs/graphocr/lib/python3.10/site-packages']


In [3]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "5"


import torch
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


from collections import defaultdict
from datasets import Collator

import os

from torchvision.datasets import CIFAR10, CIFAR100, ImageFolder
from torch.utils.data import DataLoader

from transformers import ViTForImageClassification,  CLIPModel, ViTModel, ViTImageProcessor, AutoModel, AutoModelForImageClassification, Dinov2ForImageClassification, AutoImageProcessor
from models.ode_transformer_gpt  import ViTNeuralODE
import numpy as np
import scienceplots

from torchdiffeq import odeint
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from matplotlib.lines import Line2D
import seaborn as sns
from sklearn.manifold import TSNE

from PIL import Image


cuda


In [4]:
DATASET_PATH = "/data/users/cboned/cifar"#imagenet100"
train_dataset = CIFAR100(root=DATASET_PATH, download=False, train=True) ## Modify to Cifar100
validation_dataset = CIFAR100(root=DATASET_PATH, download=False, train=False)

#train_dataset = ImageFolder(root=DATASET_PATH + "/train")
#validation_dataset = ImageFolder(root=DATASET_PATH + "/val")

In [5]:
processor = AutoImageProcessor.from_pretrained('facebook/dino-vitb16', use_fast=False)
collator = Collator(processor)

train_dloader = DataLoader(train_dataset, 
                           shuffle=False,
                           batch_size=1,
                           pin_memory=False,
                           num_workers=1,
                           collate_fn=collator.classification_collate_fn
                           )


test_dloader = DataLoader(validation_dataset,
                          batch_size=1,
                          pin_memory=False,
                          num_workers=1,
                          drop_last=True,
                          shuffle=True,
                          collate_fn=collator.classification_collate_fn

)

In [6]:
model = ViTModel.from_pretrained("facebook/dino-vitb16").to(device)
model = model.eval()
model.set_attn_implementation('eager')

Some weights of ViTModel were not initialized from the model checkpoint at facebook/dino-vitb16 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
EDO_CHECKPOINT_PATH = "/data/users/cboned/checkpoints/EDO_DISTILLATION_VIT_ON_CIFAR100_DINO_ATTENTION_1-Attention_L1_NO_JASMIN_add_supervision_at_the_end.pt"

edo_model = ViTNeuralODE(
    img_size=224,
    patch_size=16,
    in_chans=3,
    mlp_ratio=1,
    num_classes=100,
    embed_dim=768,
    num_heads=12,
    emulate_depth=12.0,
    time_interval=1.0,   # match 12 "layers" by integrating over [0,12]
    num_eval_steps=24,
    solver="euler",
    register_tokens=10,
    pos_embed_register_tokens=False
)

edo_model.patch_embed.pos_embed = (
    model.embeddings.position_embeddings)


try:
    edo_model.load_state_dict(torch.load(EDO_CHECKPOINT_PATH, weights_only=True)["state_dict"])
except Exception as e:
    print(e)
    print("Matching individual Weights")
    weight_to_update = torch.load(EDO_CHECKPOINT_PATH, weights_only=True)
    for w in weight_to_update.keys():
        if edo_model.state_dict().get(w) is not None:
            edo_model.state_dict()[w].data.copy_(weight_to_update[w])

                
edo_model = edo_model.to(device)
edo_model = edo_model.eval()

Error(s) in loading state_dict for ViTNeuralODE:
	size mismatch for odefunc.block.mlp.fc1.weight: copying a param with shape torch.Size([3072, 768]) from checkpoint, the shape in current model is torch.Size([768, 768]).
	size mismatch for odefunc.block.mlp.fc2.weight: copying a param with shape torch.Size([768, 3072]) from checkpoint, the shape in current model is torch.Size([768, 768]).
Matching individual Weights


In [11]:
from thop import clever_format, profile
import tqdm 
input_tensor = torch.randn(64, 3, 224, 224).to(device)
fl = 0

with torch.no_grad():
    #for b in tqdm.tqdm(range(input_tensor.shape[0]), total=input_tensor.shape[0]):
    input_t = input_tensor#[b:b+1]
    
    flops, params = profile(edo_model, verbose=False, inputs=(input_t,))
    fl +=flops

fl_readable, params_readable = clever_format([fl, params], "%.3f")


In [12]:
fl, params_readable

(733694263296.0, '1.847M')

In [15]:
with torch.no_grad():
    input_tensor = torch.randn(1000, 3, 224, 224).to(device)
    flops, params = profile(model, verbose=False, inputs=(input_tensor,))

In [17]:
clever_format([flops, params], "%.3f")



('16.863T', '86.237M')

In [ ]:
(16.863-5.732)